## Import packages

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spstats

# Data Cleaning

## Print head of the data

In [8]:
data = pd.read_csv('data.csv')

data.head()

,game_id,Date,Time,PitchNo,Inning,inning_half,PAofInning,PitchofPA,Pitcher,PitcherId,...,z0,vx0,vy0,vz0,ax0,ay0,az0,catcher,catcher_id,catcher_team
0,20240220-HighPointUniversity-1,2024-02-20,60314.0,82,3,Top,4,2,"Olsovsky, Dalton",1000251274,...,5.41,3.28,-106.58,0.58,12.91,22.07,-34.04,"Ruiz, Justin",1.000209e+09,HIG_PAN
1,20240220-HighPointUniversity-1,2024-02-20,63576.0,185,6,Top,2,3,"Glover, Lucas",1000138461,...,6.01,5.23,-118.11,-3.37,3.74,26.71,-27.88,"Ruiz, Justin",1.000209e+09,HIG_PAN
2,20240220-HighPointUniversity-1,2024-02-20,66446.0,269,8,Top,3,1,"Carter, Noah",1000108939,...,5.52,3.08,-122.94,-1.77,-3.68,24.28,-16.89,"Grintz, Eric",6.864560e+05,HIG_PAN
3,20240220-HighPointUniversity-1,2024-02-20,64809.0,216,6,Bottom,6,1,"Welch, Collin",1000192105,...,6.23,0.38,-117.74,-0.37,-12.27,24.59,-28.63,"Church, Braxton",1.000192e+09,APP_MOU
4,20240220-HighPointUniversity-1,2024-02-20,67985.0,308,9,Bottom,4,3,"Lewis, Zach",1000127413,...,5.18,9.62,-132.57,-10.20,-8.59,32.05,-14.93,"Church, Braxton",1.000192e+09,APP_MOU


# Data Exploration

# Feature Engineering